<a href="https://colab.research.google.com/github/pedrohenrocha/data_processing_module/blob/main/spark/examples/06-write_partitioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Write
- .write
- .format (parquet, csv, json)
- options
- spark.sql.sources.partitionOverwriteMode dynamic

# Write Mode
- overwrite - The overwrite mode is used to overwrite the existing file, alternatively, you can use SaveMode.Overwrite
- append - To add the data to the existing file, alternatively, you can use SaveMode.Append
- ignore - Ignores write operation when the file already exists, alternatively, you can use SaveMode.Ignore.
- errorifexists or error - This is a default option when the file already exists, it returns an error, alternatively, you can use SaveMode.ErrorIfExists.

# Partitioning
Process to organize the data into multiple chunks based on some criteria.
Partitions are organized in sub-folders.
Partitioning improves performance in Spark.

# Setting up PySpark

In [ ]:
%pip install pyspark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Spark Course').getOrCreate()

# Preparing data

In [2]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.3 MB/s eta 0:00:00


In [3]:
from faker import Faker
from datetime import datetime

fake = Faker()

users = []
for _ in range(50):
    user = {
        'date': fake.date_time_between_dates(datetime(2024, 5, 1), datetime(2024, 5, 5)),
        'name': fake.name(),
        'address': fake.address(),
        'email': fake.email(),
        'dob': fake.date_of_birth(),
        'phone': fake.phone_number()
    }
    users.append(user)

df = spark.createDataFrame(users)

df.show(10, False)


+--------------------------------------------------------------+--------------------------+----------+--------------------------+----------------+----------------------+
|address                                                       |date                      |dob       |email                     |name            |phone                 |
+--------------------------------------------------------------+--------------------------+----------+--------------------------+----------------+----------------------+
|59077 Joshua Ridges\nPort Julie, OK 99092                     |2024-05-03 09:01:14.749509|1946-08-08|michaellewis@example.org  |Debra Conway    |(777)925-5409x40604   |
|324 Tammy Spring Apt. 966\nStevenburgh, RI 17678              |2024-05-02 00:40:15.946107|1945-02-22|ybrown@example.org        |Mary Miller     |(327)968-3327         |
|USCGC Moore\nFPO AA 13632                                     |2024-05-04 21:35:43.139512|1934-02-12|sandovaltodd@example.org  |Amanda Nelson   |001-

# Writing as PARQUET



In [4]:
# Writing as PARQUET with no partitions

path = "/content/write_partitioning/parquet_no_partitions"

df.write.mode("overwrite").format("parquet").save(path)

!ls /content/write_partitioning/parquet_no_partitions

spark.read.format("parquet").load(path).count()

part-00000-96ecb57c-53dd-4d68-adc2-6dfad77f4f4d-c000.snappy.parquet  _SUCCESS


50

In [5]:
# Writing as PARQUET with partitions
from pyspark.sql.functions import *

path = "/content/write_partitioning/parquet_with_partitions"

# Creating partition column
df = df.withColumn("date_part", date_format(col("date"), "yyyyMMdd"))

spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic") # enable dynamic partition overwrite - only overwrites partitions that are coming in the dataframe

(df#.where("date_part = '20240503'")
 .write
 .mode("overwrite")                                               # overwrites the entire path with the new data
 .partitionBy("date_part")                                        # partition the data by column - creates sub-folders for each partition
 .format("parquet")                                               # format of output
 .save(path))                                                     # path

!ls /content/write_partitioning/parquet_with_partitions

spark.read.format("parquet").load(path).count()

'date_part=20240501'  'date_part=20240503'
'date_part=20240502'  'date_part=20240504'


50

In [6]:
# Checking single partition
spark.read.parquet("/content/write_partitioning/parquet_with_partitions/date_part=20240502").show()

+--------------------+--------------------+----------+--------------------+-----------------+--------------------+
|             address|                date|       dob|               email|             name|               phone|
+--------------------+--------------------+----------+--------------------+-----------------+--------------------+
|324 Tammy Spring ...|2024-05-02 00:40:...|1945-02-22|  ybrown@example.org|      Mary Miller|       (327)968-3327|
|6946 Victor Ridge...|2024-05-02 14:31:...|2002-06-23|rebecca34@example...|    Eric Hamilton|        269-754-4528|
|647 Steven Corner...|2024-05-02 06:39:...|1965-06-07|  nmoore@example.com|   Theresa Garcia| (625)238-4008x35587|
|347 Brooke Mounta...|2024-05-02 14:46:...|1926-08-03|loretta94@example...|    Felicia Lyons|       (979)765-8474|
|9426 James Shores...|2024-05-02 13:07:...|2017-05-20|crawforddouglas@e...|Jennifer Thompson|        489-373-5796|
|437 John Mountain...|2024-05-02 03:40:...|1966-01-16|pcunningham@examp...|Micha

# Writing as CSV

https://spark.apache.org/docs/3.5.1/sql-data-sources-csv.html

In [7]:
df.count()

50

In [8]:
path = "/content/write_partitioning/csv_no_partitioning/"

# write as csv
(df
  .write
  .format("csv")
  .mode("overwrite")
  .option("delimiter", "|")
  .option("header", True)
  .save(path))

# listing files in the folder
!ls /content/write_partitioning/csv_no_partitioning/

# read as csv
(spark
  .read
  .options(sep="|", multiLine=True, header=True)
  .csv(path)
  .count())

part-00000-ec26e66d-f5cd-4095-beb5-0864874141ad-c000.csv  _SUCCESS


50

# Writing as JSON

https://spark.apache.org/docs/3.5.1/sql-data-sources-json.html

In [9]:
path = "/content/write_partitioning/json_no_partitioning/"

# write as json
(df
.write
.mode("overwrite")
.format("json")
.save(path))

# listing files in the folder
!ls /content/write_partitioning/json_no_partitioning/

# read as json
(spark
  .read
  .json(path)
  .count())

part-00000-364a14f8-c6cb-4592-889f-ef2e9b2d8c76-c000.json  _SUCCESS


50

In [10]:
# reading json as text
spark.read.text(path).show(10, False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                         |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"address":"59077 Joshua Ridges\nPort Julie, OK 99092","date":"2024-05-03T09:01:14.749Z","dob":"1946-08-08","email":"michaellewis@example.org","name":"Debra Conway","phone":"(777)925-5409x40604","date_part":"20240503"}                    |
|{"address":"324 Tammy Spring Apt. 9

In [11]:
# reading json as text
spark.read.json(path).show(10, False)

+--------------------------------------------------------------+------------------------+---------+----------+--------------------------+----------------+----------------------+
|address                                                       |date                    |date_part|dob       |email                     |name            |phone                 |
+--------------------------------------------------------------+------------------------+---------+----------+--------------------------+----------------+----------------------+
|59077 Joshua Ridges\nPort Julie, OK 99092                     |2024-05-03T09:01:14.749Z|20240503 |1946-08-08|michaellewis@example.org  |Debra Conway    |(777)925-5409x40604   |
|324 Tammy Spring Apt. 966\nStevenburgh, RI 17678              |2024-05-02T00:40:15.946Z|20240502 |1945-02-22|ybrown@example.org        |Mary Miller     |(327)968-3327         |
|USCGC Moore\nFPO AA 13632                                     |2024-05-04T21:35:43.139Z|20240504 |1934-02-12|

In [12]:
# partition json data + saveAsTable

# Creating partition column
df = df.withColumn("date_part", date_format(col("date"), "yyyyMMdd"))

# write as json
(df.write
  .partitionBy("date_part")
  .mode("overwrite")
  .format("json")
  .saveAsTable("tbl_json_part"))

# read as json
spark.table("tbl_json_part").count()

# read as json
spark.sql("show partitions tbl_json_part").show()

+------------------+
|         partition|
+------------------+
|date_part=20240501|
|date_part=20240502|
|date_part=20240503|
|date_part=20240504|
+------------------+



# Append Mode

In [13]:
# Writing as PARQUET with APPEND

path = "/content/write_partitioning/parquet_append"

df.write.mode("append").format("parquet").save(path)

!ls /content/write_partitioning/parquet_append

spark.read.format("parquet").load(path).count()

part-00000-21fcff2a-0e6c-49e7-b02a-3338debad347-c000.snappy.parquet  _SUCCESS


50